# Data process

In [1]:
import pandas as pd
import numpy
df = pd.read_csv('AB_test_data.csv')

In [2]:
df_A = df[df['Variant'] == 'A']
df_B = df[df['Variant'] == 'B']

# test1

In [3]:
t = (df_B['purchase_TF'].mean() - df_A['purchase_TF'].mean()) / numpy.sqrt((df['purchase_TF'].mean() * (1-df['purchase_TF'].mean())) * (1 / (len(df_B['purchase_TF'])+ 1 / len(df_A['purchase_TF']))))
print(t)

5.334071792564316


# test2

In [9]:
p0 = df_A['purchase_TF'].mean()
p1 = df_B['purchase_TF'].mean()
t_a = 1.645
t_b = 0.84
p_mean = df['purchase_TF'].mean()
min_effect = abs(p1 - p0)
n = ((t_a) * numpy.sqrt(2 * p_mean * (1 - p_mean)) + t_b * numpy.sqrt( p0 * (1 - p0) + p1 * (1 - p1))) ** 2 / (min_effect * min_effect)
n = int(n) + 1
print('The sample size is: %d.'%n)

The sample size is: 2218.


In [18]:
sample = []
for i in range(10):
    sample_A =  df_A.sample(n)
    sample_B =  df_B.sample(n)
    t1 = (sample_B['purchase_TF'].mean() - sample_A['purchase_TF'].mean()) / numpy.sqrt((sample_A['purchase_TF'].var()+sample_B['purchase_TF'].var()) / n)
    sample.append([sample_A, sample_B])
    if t1 > t_a:
        print('reject H0')
    else:
        print('accept H0')

reject H0
reject H0
reject H0
reject H0
reject H0
reject H0
reject H0
accept H0
reject H0
reject H0


# test 3

# conservative

In [30]:
boundary_wald_low = numpy.log(0.2 / (1 - 0.05))
boundary_wald_high = numpy.log((1 - 0.2) / 0.05)
boundary_conservative_low = numpy.log(0.2)
boundary_conservative_high = numpy.log(1 / 0.05)
f_1 = numpy.log(p1 / p0)
f_0 = numpy.log((1 - p1) / (1 - p0))
count_list = []
for j in range(10):
    summary = 0
    count = 0
    sample_A = sample[j][0]
    sample_B = sample[j][1]
    sample_combine = pd.concat([sample_A, sample_B])
    sample_random = sample_combine.sample(frac=1.0)
    for i in range(len(sample_random)):
        count += 1
        if sample_random['purchase_TF'].values[i] == 1:
            summary += f_1
            if summary > boundary_conservative_high:
                print('stop early, reject H0')
                count_list.append(count)                
                break
        else:
            summary += f_0
            if summary < boundary_conservative_low:
                print('stop early, accept H0')
                count_list.append(count) 
                break
        if count == len(sample_random):
            print('use the full sample')
            count_list.append(count)
total = 0
for i in range(10):
    total += count_list[i]
average = total / 10
print('the average number of iterations is: %f.'%average)

stop early, accept H0
stop early, accept H0
stop early, reject H0
stop early, accept H0
stop early, accept H0
stop early, reject H0
stop early, accept H0
stop early, reject H0
stop early, accept H0
stop early, accept H0
the average number of iterations is: 714.800000.


# wald

In [37]:
count_list = []
for j in range(10):
    summary = 0
    count = 0
    sample_A = sample[j][0]
    sample_B = sample[j][1]
    sample_combine = pd.concat([sample_A, sample_B])
    sample_random = sample_combine.sample(frac=1.0)
    for i in range(len(sample_random)):
        count += 1
        if sample_random['purchase_TF'].values[i] == 1:
            summary += f_1
            if summary > boundary_wald_high:
                print('stop early, reject H0')
                count_list.append(count)                
                break
        else:
            summary += f_0
            if summary < boundary_wald_low:
                print('stop early, accept H0')
                count_list.append(count) 
                break
        if count == len(sample_random):
            print('use the full sample')
            count_list.append(count)
total = 0
for i in range(10):
    total += count_list[i]
average = total / 10
print('the average number of iterations is: %f.'%average)

stop early, reject H0
stop early, accept H0
stop early, reject H0
stop early, reject H0
stop early, reject H0
stop early, reject H0
stop early, accept H0
stop early, accept H0
stop early, accept H0
stop early, reject H0
the average number of iterations is: 622.400000.
